### Note:
In this directory, there are two examples using PDFs: **Example 3 - Plot PDF for a station**, and **Example 4 - Calculate PDFs from PSDs**.  These two examples are provided in order to highlight different ways that you can use the PDF and PSD values that ISPAQ generates.  

To be specific, the difference between the two examples are:  

#### Example 3 - Plot PDF for a station:  
Example 3 uses PDFs that already exist in the ISPAQ example database. This means that they have been calculated using an ispaq.py command with the `--output db --db_name ispaq_example.db` options.  

This is a great way to do it, especially if you plan to run the PSDs and PDFs at the same time, say on some sort of regular schedule.  In that case, you might as well calculate both in the same command and store them both in the ISPAQ database for later retrieval.  

Additionally, we have tried to make it simple to calculate PDFs in ISPAQ for cases where you already have PSDs for the time span you are interested in.  For example, PDFs calculation does not require seismic data since it instead reads in existing PSDs.  That means that if you, the user, have been calculating daily PSDs for the past year, you don’t need to load a year’s worth of data to calculate a year-long PDF  - you can just use the existing PSDs! By calculating that year-long PDF using ISPAQ, it will be saved to either the database or the csv file and you will be able to retrieve it later.  

#### Example 4 - Calculate PDFs from PSDs:  
Example 4 will calculate PDFs _on the fly_, meaning that they do not need to exist in the ISPAQ metric database, nor will they be saved to the ISPAQ metric database.   

Why would you want to do this if you can simply use an ispaq.py command to calculate and save the PDFs in the database?  Here are a couple possible reasons: 
1) You may want to calculate PDFs on an arbitrary timeframe but don't feel the need to save the PDF values, say if you are just poking around at or investigating changes in the data and don't want to clutter the database.   

2) To prevent the ISPAQ database from growing too complicated, the pdf table in the ISPAQ database is very simple and PDFs values are stored with the start and end times used to calculate that particular PDF.  If you calculate daily PDFs for a week and then additionally calculate a week-long PDF, the database will store 8 PDFs - one for each day in the week, and one that spans the entire week. This means that, even if you have used ISPAQ to calculate your arbitrary time frame, you must know the specific start and end times of the PDF that you are looking to retrieve. If you look for a time range using less-than and greater-than (<>) instead of equals-to (==) then you risk retrieving multiple PDFs, including ones that you did not intend. By using this on-the-fly method, you bypass this risk since PSDs are stored by the individual PSD (usually an hour span, can vary depending on the sample rate of the data), and only those PSDs that are needed to calculate the PDF are retrieved. 




*Both methods are valid and can be useful in different situations.*

## Example 3 - Plot PDF for a station
The intent of this series of Jupyter Notebooks is to demonstrate how metrics can be retrieved from the ISPAQ example sqlite database and provide some ideas on how to  use or plot those metrics.  

This example creates a PDF plot for a station. It requires that we have the PDF values already calculated for the target for the requested
days, and those values should live in the ISPAQ example database. To generate PDFs, corrected PSD values must already exist. If they do not yet exist, then you can run them via (this will take several minutes):

    ./run_ispaq.py -M psd_corrected -S ANMO --starttime 2020-10-01 --endtime 2020-10-16 --output db --db_name ispaq_example.db

To calculate PDF values:

    ./run_ispaq.py -M pdf -S ANMO --starttime 2020-10-01 --endtime 2020-10-16 --output db --db_name ispaq_example.db --pdf_interval aggregated

Note: The above command will also create a PDF plot if the `pdf_type` parameter is set to 'plot' in the preference file or on the command line. This Jupyter tutorial demonstrates how to create custom PDF plots without recalculating the PDF values. This plot has a different color scheme from the default plots and does not include the noise model or the max/mode/min curves.

Or to calculate both PSDs and PDFs at the same time:

    ./run_ispaq.py -M psd_corrected,pdf -S ANMO --starttime 2020-10-01 --endtime 2020-10-16 --output db --db_name ispaq_example.db --pdf_interval aggregated

This example will assume that the above command has already been run and the PDFs already exist in the database.

To begin, we need to import the necessary modules:

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
import numpy as np
import datetime

Because PDFs are calculated for set frequency bins, which depend on the sample rate of the data, we create a simple function that will help us with placing our tick marks in the right location in the plot.

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

And now set some variables.

In [ ]:
db_name = '../ispaq_example.db'
metric = 'pdf'
startDate = '2020-10-01T00:00:00.000000Z'       # Full time is important for retrieving PDFs
endDate = '2020-10-15T23:59:59.000000Z'
target = 'IU.ANMO.00.BH1.M'

startdate = startDate.split('T')[0]
enddate = endDate.split('T')[0]
filename = f'example3_{target}_{startdate}_{enddate}_PDF.png'

The first step is to create a query that will be used to retrieve the PDFs.

In [ ]:
SQLcommand = f"SELECT * FROM {metric} WHERE start = '{startDate}' " \
             f"and end = '{endDate}' and (target = '{target}');"
print(SQLcommand)

Create a connection to the database and run the query, loading it into a pandas dataframe

In [ ]:
try:
    conn = sqlite3.connect(db_name)
    DF = pd.read_sql_query(SQLcommand, conn, parse_dates=['start','end'])
    conn.close
except:
    print(f"Unable to connect to or find the {metric} table in the database {db_name}")

In [ ]:
if DF.empty:
    print("Empty return: there are no PDFs that were retrieved")

In [ ]:
print(DF)

Sum up the total number of hits for each frequency:

In [ ]:
for frequency in DF['frequency'].unique():
        # Sum hits for total column
        DF.loc[DF['frequency'] == frequency, 'total'] = sum(DF[DF['frequency'] == frequency]['hits'])

For each frequency-power bin, calculate what percentage of the total hits for that frequency are at that power. 

In [ ]:
DF['percent'] = DF['hits'] / DF['total'] * 100

Create a minimum range of powers (Y-axis) for better viewing.

In [ ]:
p1 = int(min(DF['power'].unique()))
p2 = int(max(DF['power'].unique()))
if p1 > -190:
    p1 = -190
if p2 < -90:
    p2 = -90
    
powers = sorted(range(p1,p2+1), reverse=True)
freqs = sorted(DF['frequency'].unique(),reverse = True)

Create a new dataframe for plotting: rows are powers, columns are periods, value is percent of hits

In [ ]:
plotDF = pd.DataFrame(0,index=powers,columns=freqs)
nonZeroFreqs=[]
for power in powers:
    for freq in freqs:
        value = DF[(DF['frequency']==freq) & (DF['power']== power)]['percent'].values
        try:
            plotDF.loc[power,freq] = value[0]
            if value[0] != 0:
                # Keep track of the frequencies that have hits, for axes limits
                nonZeroFreqs.append(freq)
        except:
            continue

Matplotlib imshow takes a list (matrix) of values, so convert the dataframe to a list

In [ ]:
plotList = plotDF.values.tolist()

And now we set up some plotting options:

In [ ]:
# Set up plotting -- color map
cmap = plt.get_cmap('gist_heat', 3000)  # You can change the colormap here
cmaplist = [cmap(i) for i in range(cmap.N)]

# convert the first nchange to fade from white, so that anywhere without any hits (or very few) is white
nchange = 100
for i in range(nchange):
    first = cmaplist[nchange][0]
    second = cmaplist[nchange][1]
    third = cmaplist[nchange][2]
    scaleFactor = (nchange-1-i)/float(nchange)

    df = ((1-first) * scaleFactor) + first
    ds = ((1-second)* scaleFactor) + second
    dt = ((1-third) * scaleFactor) + third

    cmaplist[i] = (df, ds, dt, 1)

cmaplist[0] = (1,1,1,1)
cmap = cmap.from_list('Custom cmap', cmaplist, cmap.N)

In [ ]:
# Set up plotting -- axis labeling and ticks
periodPoints = [0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]
freqPoints = [1/float(i) for i in periodPoints]
xfilter = [(i <= freqs[0]) and (i >= freqs[-1]) for i in freqPoints]
xlabels = [i for (i, v) in zip(freqPoints, xfilter) if v]
xticks = [find_nearest(freqs, i) for i in xlabels]
xlabels = [int(1/i)  if i<=1 else 1/i for i in xlabels]     #convert to period, use decimal only if <1s

yticks = [powers.index(i) for i in list(filter(lambda x: (x % 10 == 0), powers))]
ylabels = [powers[i] for i in yticks]

In [ ]:
# Set up plotting -- plot
height = ylabels[0] - ylabels[-1]
plt.figure(figsize=( 12, (.055*height + .5) ))

In [ ]:
plt.imshow(plotList, cmap=cmap,  vmin=0, vmax=30, aspect=.4, interpolation='bilinear')
# Adjust grids, labels, limits, titles, etc
plt.grid(linestyle=':', linewidth=1)
plt.xlabel('Period (s)',size=18)
plt.ylabel(r'Power [$10log_{10}(\frac{m^2/s^4}{hz}$)][dB]',size=18)

plt.xticks(xticks[::-1], xlabels[::-1],size=15)
plt.yticks(yticks,ylabels,size=15)

xmin=freqs.index(min(nonZeroFreqs))
xmax=freqs.index(max(nonZeroFreqs))
plt.xlim(xmax,xmin)
plt.ylim(max(yticks)+5,min(yticks)-5)

plt.title(f"{target}\n{startdate} through {enddate}", size=18)

# User has option to include colorbar and/or legend
cb = plt.colorbar(fraction=.02)
cb.set_label('percent probability',labelpad=5)


Save the figure for later use:

In [ ]:
plt.tight_layout()
plt.savefig(filename)